In [1]:
import argparse
import random
import string
import openai
from dotenv import load_dotenv
load_dotenv()
from retry import retry


import re
import os
import sys
sys.path.insert(0, '../WebShop')
from web_agent_site.envs.web_agent_text_env import WebAgentTextEnv

/home/robert1003/miniconda3/envs/webshop/lib/python3.8/site-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/home/robert1003/miniconda3/envs/webshop/lib/python3.8/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
2023-05-23 04:33:36.974884: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 04:33:37.166164: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from

In [2]:
# Setup environment
env = WebAgentTextEnv(
        observation_mode='text_rich',
        render=False,
        num_products=None, # 1000 for small product space, None for full product space
    )

Products loaded.

Keys cleaned.

Attributes loaded.

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1181436/1181436 [00:16<00:00, 70858.66it/s]


Loaded 11674685 goals.


/home/robert1003/.local/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [3]:
openai.api_key = # fill API KEY here

@retry(tries=1, delay=20)
def gpt3(examples, prompt, max_tokens=100, stop=["\n\n"]):
    example_with_prompt = '\n===\n'.join(examples) + '\n===\n\n' + prompt.strip()

#     manual testing
     '''response = ''
     while True:
         s = input('continue?')
         if s != '':
             response += s.strip() + '\n'
         else:
             break
     return response.strip()'''
    
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=example_with_prompt,
      temperature=0,
      max_tokens=max_tokens,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response["choices"][0]["text"]

In [4]:
def ob2summary(ob, examples, llm):
    try:
        while True:
            try:
                summary = llm(examples, ob, max_tokens=500, stop=['\n\n']).strip()
                break
            except Exception as e:
                print('ob2summary exception:', e)
                cont = input('continue retry?')
                if cont == '':
                    raise Exception(e)

        template = r"info\[[\S\s]+\]"
        for res in re.finditer(template, summary):
            info = res.group()
            if info.startswith('info['):
                info = info[5:]
            if info.endswith(']'):
                info = info[:-1]

            if '[Search]' in info:
                idx = info.find('[Search]')
                info = info[:idx+len('[Search]')]

            if '[Buy Now]' in info:
                idx = info.find('[Buy Now]')
                info = info[:idx+len('[Buy Now]')]

            return info, None

        return None, f'no summary found in {summary}'
    except Exception as e:
        raise Exception(e)
        return None, e

def summary2act(traj, examples, llm, env_info):
    try:
        while True:
            try:
                act = llm(examples, traj, stop=['\n\n']).strip()
                break
            except Exception as e:
                print('summary2act exception:', e)
                cont = input('continue retry?')
                if cont == '':
                    raise Exception(e)

        available_acts = ['search', 'click', 'think']
        template = '(' + '|'.join(available_acts) + r")\[[^\[\]]+\]"
        for res in re.finditer(template, act):
            act_str = res.group()
            if act_str.startswith('think'):
                return act_str, None
            elif act_str.startswith('search'):
                if env_info['has_search_bar']:
                    return act_str, None
            elif act_str.startswith('click'):
                if act_str.lower() in list(map(lambda x: f'click[{x}]', env_info['clickables'])):
                    return act_str, None

        return None, f'no act found in {act}'
    except Exception as e:
        raise Exception(e)
        return None, e

In [5]:
# Get summary examples
with open('../prompts/ash-summarizer.txt', 'r') as f:
    summary_examples = f.read().split('\n===\n')
    
# Get summary prompt template
with open('../prompts/ash-summarizer-template.txt', 'r') as f:
    summary_prompt_template = f.read().strip()

# Get action examples
with open('../prompts/ash-actor.txt', 'r') as f:
    act_examples = f.read().split('\n===\n')

llm = gpt3
max_fail = 5
max_iter = 20
seed = 814
    
need_summary = True
done = False
counter = 0
invalid_counter = 0
traj = ''
rew = 0
term_status = None

random.seed(seed)
session = ''.join(random.choices(string.ascii_lowercase, k=10))
(ob, _) = env.reset(session=session)
info = env.get_available_actions()
act = 'None'

log_file = open('./log', 'w')

while True:
    # Get summary
    ob = '\n'.join(ob.strip().split('\n\n'))
    print(f'Observation {counter}:\n{ob}')
    print(f'Observation {counter}:\n{ob}', file=log_file)    

    if need_summary:
        summary_prompt = summary_prompt_template.format(
            action=act, observation=ob)
        summary, err = ob2summary(summary_prompt, summary_examples, llm)
    else:
        summary, err = ob, None
    if err is not None:
        raise Exception(f'Error in ob2summary() of trajectory {seed}:', err)
    print(f'Summary {counter}:\n{summary}')
    print(f'Summary {counter}:\n{summary}', file=log_file)    

    if counter == 0:
        traj += f'Instruction: ' + summary.strip()
    else:
        traj += f'\nObservation {counter}:\n' + summary.strip()

    # Get action
    act, err = summary2act(traj + f'\n\nAction {counter+1}: ', act_examples, llm, info)
    if err is not None and 'no act found' not in err:
        raise Exception(f'Error in summary2act() of trajectory {seed}:', err)

    # Case by case analysis for action
    if err is not None and 'no act found' in err:
        ob = 'Invalid Action.'
        need_summary = False
        invalid_counter += 1
        if invalid_counter >= max_fail:
            rew = 0
            term_status = 'max_fail'

        print(f'Action {counter+1}: {act} (invalid)')
        print(f'Action {counter+1}: {act} (invalid)', file=log_file)
    else:
        invalid_counter = 0
        if act.startswith('think'):
            ob = 'OK.'
            need_summary = False
        else:
            ob, rew, done, _ = env.step(act)
            info = env.get_available_actions()
            need_summary = True

        traj += f'\n\nAction {counter+1}: ' + act.strip()
        print(f'\nAction {counter+1}: {act}')
        print(f'\nAction {counter+1}: {act}', file=log_file)

        if done:
            break

    # terminate if max_iter reached
    counter += 1
    if counter >= max_iter:
        rew = 0
        term_status = 'max_iter'
        
print('reward', rew, 'term_status', term_status)
print('reward', rew, 'term_status', term_status, file=log_file)

Observation 0:
WebShop
Instruction: 
Find me twin size bedframes with box spring, memory foam with pattern name: bed frame + mattress, 12 inch, and size: queen, and style: 10 inch, and price lower than 130.00 dollars
[button] Search [button_]
Summary 0:
Find me twin size bedframes with box spring, memory foam with pattern name: bed frame + mattress, 12 inch, and size: queen, and style: 10 inch, and price lower than 130.00 dollars.
[Search]

Action 1: search[twin size bedframes with box spring memory foam pattern name bed frame + mattress 12 inch queen style 10 inch]
Observation 1:
Instruction:
Find me twin size bedframes with box spring, memory foam with pattern name: bed frame + mattress, 12 inch, and size: queen, and style: 10 inch, and price lower than 130.00 dollars
[button] Back to Search [button_]
Page 1 (Total results: 50)
[button] Next > [button_]
[button] B07R12845W [button_]
Linenspa 8 Inch Memory Foam and Innerspring Hybrid Mattress with Linenspa 14 Inch Folding Platform Bed